In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, Attention
from tensorflow.keras.models import Model
import re
import numpy as np
from tensorflow.keras.models import load_model

import pandas as pd
import glob

2023-11-22 18:42:08.727492: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 18:42:09.067593: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 18:42:09.067642: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 18:42:09.069679: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 18:42:09.271552: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 18:42:09.274021: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [8]:
import pandas as pd
import re
import glob

file_pattern = f"data/conve*.txt"
def extract_number(file_name):
    match = re.search(r'\d+', file_name)
    return int(match.group()) if match else None

# Filter files based on the extracted number within the range of 0 to 50
filtered_files = [file for file in matching_files if 0 <= extract_number(file) <= 50]
matching_files = glob.glob(file_pattern)
print(len(filtered_files))
print(len(matching_files))
input_texts = []
output_texts = []
df = pd.DataFrame()
for file_path in filtered_files:
    with open(file_path, "r") as file:
        content = file.read()

    examples = content.split("assistant: ")

    inputs = []
    outputs = []

    i = 1
    for example in examples[1:]:
        for _ in range(100):
            input_match = re.search(f"Input{i}: (.+?)\n", example)
            output_match = re.search(f"Output{i}: (.+?)\n", example)

            if input_match and output_match:
                # outputs_value = "flowchart TD \n\t" + output_match.group(1)
                outputs_value = output_match.group(1)
                # outputs_value = format_mermaid_code(outputs_value)
                inputs_value = re.sub(r'Output.*', r'', input_match.group(1))
                inputs.append(inputs_value)
                outputs.append(outputs_value)
                output_texts.append(outputs_value)
                input_texts.append(inputs_value)
            i += 1  
    newDf = pd.DataFrame({"input": inputs, "output": outputs})
    df = pd.concat([df, newDf], ignore_index=True)


50
128


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.models import Model
import numpy as np

# Define the maximum sequence length for input and output sequences
max_input_seq_len = max(len(seq) for seq in input_texts)
max_output_seq_len = max(len(seq) for seq in output_texts)

# Define the input and output tokenizers
input_tokenizer = Tokenizer(filters='', char_level=False)
output_tokenizer = Tokenizer(filters='', char_level=False)

# Fit the input tokenizer on the preprocessed input sequences
input_tokenizer.fit_on_texts(input_texts)
print(len(input_tokenizer.word_index))
# Fit the output tokenizer on the preprocessed output sequences
output_tokenizer.fit_on_texts(output_texts)

# Convert the input and output sequences to numerical format
encoder_input_seqs = input_tokenizer.texts_to_sequences(input_texts)
decoder_input_seqs = output_tokenizer.texts_to_sequences(output_texts)
decoder_target_seqs = [seq[1:] for seq in decoder_input_seqs]

# Pad the input and output sequences to fixed length
encoder_input_seqs = pad_sequences(encoder_input_seqs, maxlen=max_input_seq_len, padding='post')
decoder_input_seqs = pad_sequences(decoder_input_seqs, maxlen=max_output_seq_len, padding='post')
decoder_target_seqs = pad_sequences(decoder_target_seqs, maxlen=max_output_seq_len, padding='post')

# Define the model architecture
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_input_seq_len,))
encoder_embedding = Embedding(input_dim=len(input_tokenizer.word_index) + 1, output_dim=latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_sequences=True, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_output_seq_len,))
decoder_embedding = Embedding(input_dim=len(output_tokenizer.word_index) + 1, output_dim=latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Attention layer
attention = Attention()([decoder_outputs, encoder_lstm])
context = Concatenate(axis=-1)([attention, decoder_outputs])

# Dense layer for prediction
decoder_dense = Dense(len(output_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(context)




1033


# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs, epochs=10, batch_size=1)

# Save the model
model.save('my_model_latent_dim_64.h5')


In [22]:

# Load the trained model
model = load_model('my_model_latent_dim_16_retrained3.h5')

# Define the input sentence
input_sentence = "Scan a list of prices until finding the one with the highest value"

# Preprocess the input sentence
input_sequence = input_tokenizer.texts_to_sequences([input_sentence])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_input_seq_len, padding='post')

# Initialize the decoded sequence
decoded_sequence = np.zeros((1, max_output_seq_len))

# Use the model to predict the output sequence
for i in range(max_output_seq_len):
    predictions = model.predict([padded_input_sequence, decoded_sequence])
    predicted_token_index = np.argmax(predictions[0, i, :])
    decoded_sequence[0, i] = predicted_token_index

    if predicted_token_index == 0:  # End of sequence token
        break

# Convert the predicted sequence back to text
output_sentence = output_tokenizer.sequences_to_texts([decoded_sequence[0]])

print("Input sentence:", input_sentence)
print("Predicted output:", output_sentence)


2023-11-20 22:58:37.464953: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 1800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


1/1 [==============================] - 0s 68ms/step
Input sentence: Scan a list of prices until finding the one with the highest value
Predicted output: ['f[end] up your a b --> a new --> you new list] b --> a b -- no --> b c -- yes --> b c -- yes --> b c -- yes --> b c -- yes --> b']


In [21]:
# RETRAINING A NEW MODEL
existing_model = load_model('my_model_latent_dim_16.h5')
existing_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
existing_model.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs, epochs=10, batch_size=1)
existing_model.save('my_model_latent_dim_64.h5')

Epoch 1/10


2023-11-20 21:22:27.627862: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 1800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


2293/2293 [==============================] - 581s 252ms/step - loss: 3.4928 - accuracy: 0.4325
Epoch 2/10
2293/2293 [==============================] - 566s 247ms/step - loss: 3.1967 - accuracy: 0.4574
Epoch 3/10
2293/2293 [==============================] - 565s 247ms/step - loss: 2.9853 - accuracy: 0.4779
Epoch 4/10
2293/2293 [==============================] - 567s 247ms/step - loss: 2.8127 - accuracy: 0.4947
Epoch 5/10
2293/2293 [==============================] - 562s 245ms/step - loss: 2.6676 - accuracy: 0.5065
Epoch 6/10
2293/2293 [==============================] - 561s 245ms/step - loss: 2.5374 - accuracy: 0.5198
Epoch 7/10
2293/2293 [==============================] - 559s 244ms/step - loss: 2.4211 - accuracy: 0.5340
Epoch 8/10
2293/2293 [==============================] - 560s 244ms/step - loss: 2.3187 - accuracy: 0.5451
Epoch 9/10
2293/2293 [==============================] - 563s 245ms/step - loss: 2.2252 - accuracy: 0.5559
Epoch 10/10
2293/2293 [==============================] - 

In [18]:
model = load_model('my_model_latent_dim_16.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 420)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 653)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 420, 16)              70608     ['input_3[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, 653, 16)              123488    ['input_4[0][0]']             
                                                                                              